# Objective

Download the data for gaging station based on the USGS site number and plot the result.

In [1]:
def get_station_data(site_no="0702200"):
    # importing the requests library 
    import requests 
    import xml.etree.ElementTree as ET
    from zipfile import ZipFile
    import io
    from io import BytesIO
    # defining the api-endpoint  
    URL = "https://cida.usgs.gov/"
    API_ENDPOINT = "https://cida.usgs.gov/sediment/data"
    SITE_NO = "07022000"
    ogcfilter = ET.Element("ogc:Filter",attrib={"xmlns:ogc":"http://www.opengis.net/ogc"})
    property_is = ET.SubElement(ogcfilter, "ogc:PropertyIsEqualTo", attrib={"matchCase":"true"})
    ET.SubElement(property_is, "ogc:PropertyName").text = "SITE_NO"
    ET.SubElement(property_is, "ogc:Literal").text = SITE_NO
    ogcfilter_string = ET.tostring(ogcfilter).decode()  
    data = {'email' : 'user@usace.army.mil',
            'format': 'tsv',
            'directDownload':'true',
            'dataTypes': 'sites_data_daily_discrete_',
            'dailyFilter' : ogcfilter_string,
            'discreteFilter' : ogcfilter_string
           }

    # sending post request and saving response as response object 
    search = requests.post(url = API_ENDPOINT, data = data, allow_redirects=True)  
    download = requests.get(url=URL+search.text)
    zipfile = ZipFile(BytesIO(download._content))
    return zipfile
zipfile = get_station_data()

In [2]:
def get_comments(filename):
    ncomments=0
    comments=''
    with zipfile.open(filename) as f:
        commentline = f.readline().decode()
        while commentline:
            if commentline[0] == '#' or commentline[:2] == r'"#':
                ncomments +=1
                comments += commentline.strip(r'"').strip('#')
                commentline=f.readline().decode()
            else:
                commentline=None
    return (ncomments, comments)
(ncomments, comments) = get_comments('discrete_data.tsv')

In [3]:
import pandas
discrete_data = pandas.read_table(zipfile.open('discrete_data.tsv'),header=ncomments,sep="\t").dropna(axis=1,how='all')
drop_columns=[]
for k in discrete_data.columns:
    if k not in ['DATETIME','SSC','DAILYFLOW','P63']:
        drop_columns.append(k)
discrete_data.drop(drop_columns,axis=1).head()

,DATETIME,SSC,DAILYFLOW,P63
0,1973-01-30 10:30:00,306,369000,NaN
1,1973-02-21 10:50:00,436,237000,NaN
2,1973-06-21 11:00:00,468,408000,NaN
3,1973-07-18 09:30:00,227,179000,NaN
4,1973-08-14 10:00:00,211,208000,NaN


In [4]:
%matplotlib ipympl
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
ssc = np.zeros(discrete_data.SSC.shape,'d')
for i,ssci in enumerate(discrete_data.SSC):
    if 'A' in ssci:
        ssc[i] = float(ssci[2:])
    else:
        ssc[i] = float(ssci)
p63 = np.zeros(discrete_data.P63.shape,'d')
for i,p63i in enumerate(discrete_data.P63):
    if type(p63i) is not float:
        if 'A' in p63i:
            p63[i] = float(p63i[2:])
        else:
            p63[i] = float(p63i)
    else:
        p63[i] = p63i
plt.figure()
plt.scatter(discrete_data.DAILYFLOW,ssc)
sc = plt.scatter(discrete_data.DAILYFLOW,p63)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
plt.figure()
date = np.array(discrete_data.DATETIME, dtype='datetime64')
p = plt.plot(date,discrete_data.DAILYFLOW)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
file = zipfile.open('discrete_data.tsv')
newfile = open('test.tsv','w')
for line in file.readlines():
    newfile.writelines(line.decode())

In [7]:
(ncomments, comments) = get_comments('daily_data.tsv')
daily_data = pandas.read_table(zipfile.open('daily_data.tsv'),header=ncomments).dropna(axis=1,how='all')
daily_data.head()

,agency_cd,site_no,datetime,DAILY_FLOW,DAILY_FLOW_QUAL,DAILY_SSC,DAILY_SSC_QUAL,DAILY_SSL,DAILY_SSL_QUAL
0,USGS,7022000,1933-04-01,191000,A,NaN,NaN,NaN,NaN
1,USGS,7022000,1933-04-02,224000,A,NaN,NaN,NaN,NaN
2,USGS,7022000,1933-04-03,257000,A,NaN,NaN,NaN,NaN
3,USGS,7022000,1933-04-04,279000,A,NaN,NaN,NaN,NaN
4,USGS,7022000,1933-04-05,298000,A,NaN,NaN,NaN,NaN


In [8]:
plt.figure()
sc = plt.scatter(daily_data.DAILY_FLOW,daily_data.DAILY_SSC)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …